In [1]:
from faker import Faker
import mysql.connector
import re
import random
from datetime import date, timedelta

# Połączenie z bazą danych
con = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

mycursor = con.cursor()

# Inicjalizacja biblioteki Faker
fake = Faker("pl_PL")  # Generowanie danych w języku polskim

def usun_polskie_znaki(tekst):
    polskie_znaki = {
        'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z',
        'Ą': 'A', 'Ć': 'C', 'Ę': 'E', 'Ł': 'L', 'Ń': 'N', 'Ó': 'O', 'Ś': 'S', 'Ź': 'Z', 'Ż': 'Z'
    }
    return ''.join(polskie_znaki.get(c, c) for c in tekst)

domeny = ["gmail.com", "yahoo.com", "outlook.com", "wp.pl", "interia.pl", "onet.pl"]

def generuj_email(imie, nazwisko):
    domena = random.choice(domeny)
    return f"{usun_polskie_znaki(imie.lower())}.{usun_polskie_znaki(nazwisko.lower())}@{domena}"

def generuj_pensje(id_stanowisko):
    if id_stanowisko == 1:  # Manager
        return random.randint(12000, 15000)
    elif id_stanowisko in [2, 3]:  # Programista, Księgowy
        return random.randint(10000, 12000)
    elif id_stanowisko == 4:  # Specjalista ds. marketingu
        return random.randint(8000, 10000)
    else:  # Konsultant ds. obsługi klienta
        return random.randint(5000, 8000)

def generuj_date_urodzenia(id_stanowisko):
    if id_stanowisko == 1:  # Manager
        return fake.date_of_birth(minimum_age=30, maximum_age=45)
    elif id_stanowisko in [2, 3, 4]:  # Programista, Księgowy, Specjalista ds. marketingu
        return fake.date_of_birth(minimum_age=25, maximum_age=45)
    else:  # Konsultant ds. obsługi klienta
        return fake.date_of_birth(minimum_age=21, maximum_age=45)

def losuj_date_zatrudnienia(id_stanowisko):
    if id_stanowisko in [1, 2]:  # Manager, Programista
        start_date = date(2022, 1, 1)
        end_date = date(2022, 1, 5)
    else:  # Pozostałe stanowiska
        start_date = date(2022, 1, 5)
        end_date = date(2022, 1, 14)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Generowanie danych dla każdego pracownika
kolejnosc_stanowisk = [1, 2, 3, 4, 5, 5, 5]  # Kolejność id_stanowisko

for id_stanowisko in kolejnosc_stanowisk:
    imie = fake.first_name()
    nazwisko = fake.last_name()
    email = generuj_email(imie, nazwisko)
    numer_telefonu = "+48 " + " ".join([
        str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)
    ])
    adres = fake.address().replace("\n", ", ")
    pensja = generuj_pensje(id_stanowisko)
    data_urodzenia = generuj_date_urodzenia(id_stanowisko)
    data_zatrudnienia = losuj_date_zatrudnienia(id_stanowisko)

    mycursor.execute("""
        INSERT INTO Pracownicy (Imie, Nazwisko, id_stanowisko, Numer_telefonu, Email, Adres, Pensja, Data_urodzenia, Data_zatrudnienia)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (imie, nazwisko, id_stanowisko, numer_telefonu, email, adres, pensja, data_urodzenia, data_zatrudnienia))

# Zatwierdź zmiany i zamknij połączenie
con.commit()
mycursor.close()
con.close()
